# asyncio를 이용한 동시성

간단한 쓰레드 구현

In [21]:
import threading
 
def sum(low, high):
    total = 0
    for i in range(low, high):
        total += i
    print("Subthread ", total)
 
t = threading.Thread(target=sum, args=(1, 100000))
t.start()
 
print("Main Thread\r")

Subthread d
 4999950000


In [22]:
# 간단한 코루틴 예제
import asyncio

async def main():
    print('wait for it', end=' ')
    await asyncio.sleep(1)
    print('.', end=' ')
    await asyncio.sleep(1)
    print('.', end=' ')
    await  asyncio.sleep(1)
    print('.', end=' ')
    await  asyncio.sleep(1)
    print(' Legendary~~')

print(main)

asyncio.run(main())

<function main at 0x106a79d08>
wait for it . . .  Legendary~~


### 예제 18-1 spinner_thread.py 스레드로 텍스트 스피너 애니메이트하기 

In [24]:
import threading
import itertools
import time
import sys

class Signal:
    go = True
    
def spin(msg, signal):
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        time.sleep(.1)
        if not signal.go:
            break    
    print(' ' * len(status), end='\r')
    
def slow_function():
    # 메인 스레드에서 아래 코드를 실행시 GIL을 해제하므로 두번째 스레드가 실행됨
    time.sleep(3)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thingking!', signal))
    print('spinner object : ', spinner)
    spinner.start() # 스레드 시작
    result = slow_function()
    signal.go = False
    spinner.join() # 스레드가 종료될 때 까지 기다림
    return result

def main():
    result = supervisor()
    print('Answer : ', result)
    
main()

spinner object :  <Thread(Thread-26, initial)>
Answer :  42


### 예제 18-2 spinner_asyncio.py 코루틴으로 텍스트 스피너 애니메이트하기

In [25]:
import asyncio
import itertools


async def spin(msg):  
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        try:
            await asyncio.sleep(.1)  # 메인루프를 잠깐 쉬게함
        except asyncio.CancelledError:  # <3>
            break
    print(' ' * len(status), end='\r')


async def slow_function():  # <4>
    # pretend waiting a long time for I/O
    await asyncio.sleep(3)  # <5>
    return 42


async def supervisor():  # <6>
    spinner = asyncio.create_task(spin('thinking!'))  # <7>
    print('spinner object:', spinner)  # <8>
    result = await slow_function()  # <9>
    spinner.cancel()  # <10>
    return result


def main():
    result = asyncio.run(supervisor())  # <11>
    print('Answer:', result)

main()

spinner object: <Task pending coro=<spin() running at <ipython-input-25-5574d85fbb40>:5>>
Answer: 42 


### halo 소개

In [29]:
from halo import HaloNotebook as Halo
success_message = 'Loading success'
failed_message = 'Loading failed'
unicorn_message = 'Loading unicorn'

spinner = Halo(text=success_message, spinner='dots')

try:
    spinner.start()
    time.sleep(1)
    spinner.succeed()
    spinner.start(failed_message)
    time.sleep(1)
    spinner.fail()
    spinner.start(unicorn_message)
    time.sleep(1)
    spinner.stop_and_persist(symbol='🦄'.encode('utf-8'), text=unicorn_message)
except (KeyboardInterrupt, SystemExit):
    spinner.stop()

Output()

Output()

Output()

## 18.2 asyncio와 aiohttp로 내려받기

- async로 만들면 어떻게 만들지 일단 맛만 보도록 하자.

In [13]:
import os 
import asyncio
import aiohttp
from bs4 import BeautifulSoup as bs

DEST_DIR = 'async_downloads/'
IMG_PREFIX = 'async'

async def save_img_async(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            print(dir(resp))
            print(resp.content())
            async with aiofiles.open('test.png', 'wb') as f:
                await f.write(resp.content)


async def aioget(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as res:
            assert res.status == 200
            return await res.text()


async def aiocontent(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as res:
            assert res.status == 200
            return await res.read()


def save_img(img, filename):
    if img and filename:
        path = os.path.join(DEST_DIR, filename)        
        with open(path, 'wb') as f:
            f.write(img)

async def get_top_banner_test():
    url = f'https://page.kakao.com/main?categoryUid=10&subCategoryUid=1000'
    content = await aioget(url)
    
    soup = bs(content, 'html.parser')
    divs = soup.findAll('div', {'class': {'topBanner'}})
    
    if (len(divs)):
        img_src = divs[0].find('img').get('src')
        print(img_src)
        file = await aiocontent(f'https:{img_src}')
        print(img_src)
        return file, 'test.png'

    
async def main():
    file, filename = await get_top_banner_test()
    save_img(file, filename)
#     async with aiohttp.ClientSession() as session:
#         async with session.get(url) as res:
            
#             if (len(divs)):
#                 img_src = divs[0].find('img').get('src')
                
asyncio.run(main())

//dn-img-page.kakao.com/download/resource?kid=c9Ecfv/hyqDp0pr4J/P8hfqqRNcxAtp8YcBzEYGk
//dn-img-page.kakao.com/download/resource?kid=c9Ecfv/hyqDp0pr4J/P8hfqqRNcxAtp8YcBzEYGk


In [14]:
import asyncio
from collections import namedtuple
from bs4 import BeautifulSoup as bs

import aiohttp

DEST_DIR = 'async_downloads/'
IMG_PREFIX = 'async'


def get_categories():
    Category = namedtuple('Category', 'uid subuid')

    cate10 = [1000, 1001, 69, 115, 116, 119, 112]
    cate11 = [1101, 1000, 86, 120, 89, 117, 87]
    cate22 = [2203, 2201, 203, 201, 2209, 2202]
    cate21 = [2103, 201, 2102]
    cate16 = [1601, 84, 51, 113, 40]

    categories = []
    categories += [Category(uid=10, subuid=subuid) for subuid in cate10]
    categories += [Category(uid=11, subuid=subuid) for subuid in cate11]
    categories += [Category(uid=16, subuid=subuid) for subuid in cate16]
    categories += [Category(uid=21, subuid=subuid) for subuid in cate21]
    categories += [Category(uid=22, subuid=subuid) for subuid in cate22]
    return categories

def strtime(timestamp):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(timestamp))


def save_img(img, filename):
    if img and filename:
        path = os.path.join(DEST_DIR, filename)       
        with open(path, 'wb') as f:
            f.write(img)

async def get_top_banner(category):
    category_url = f'https://page.kakao.com/main?categoryUid={category.uid}&subCategoryUid={category.subuid}'
    content = await aioget(category_url)
    
    soup = bs(content, 'html.parser')
    divs = soup.findAll('div', {'class': {'topBanner'}})
    
    if (len(divs)):
        img_src = divs[0].find('img').get('src')
        file = await aiocontent(f'https:{img_src}')
        print(img_src)
        return save_img(file, f'{IMG_PREFIX}-{category.uid}-{category.subuid}-topbanner.png')
    else:
        print(f'no top banner on {category.uid} in {category.subuid}')
        return None, None


async def download_many(categories):
    async with aiohttp.ClientSession() as session:
        tasks = [get_top_banner(cate) for cate in categories]
        await asyncio.gather(*tasks)
    

async def main():
    start = time.time()
    print(f'[{strtime(start)}] start!', )
    await download_many(get_categories())
    end = time.time()
    elapsed = end - start
    print(f'[{strtime(end)}] topbanner img downloaded in {elapsed:.3}s')

asyncio.run(main())

[2019-06-10 03:22:36] start!
no top banner on 22 in 2203
//dn-img-page.kakao.com/download/resource?kid=KG9Ti/hyqE7lhcPW/ZNhkJcTU3VwGrfUOhydzW1
//dn-img-page.kakao.com/download/resource?kid=c9Ecfv/hyqDp0pr4J/P8hfqqRNcxAtp8YcBzEYGk
//dn-img-page.kakao.com/download/resource?kid=bfEAxj/hyqDrKIgl9/18PopouUrfXTWBCmMfrezK
//dn-img-page.kakao.com/download/resource?kid=IqL86/hyqFeSfxfM/VKbirHLLgOeWOee8K85JK1
//dn-img-page.kakao.com/download/resource?kid=bq6Fty/hyqDvTZnuC/T1Cd60N8cMrh0cPNZ7xF4k
//dn-img-page.kakao.com/download/resource?kid=c9Ecfv/hyqDp0pr4J/P8hfqqRNcxAtp8YcBzEYGk
//dn-img-page.kakao.com/download/resource?kid=EWozV/hyuhtYz9dW/miD6izhA0j8iGnPiq9aLq0
//dn-img-page.kakao.com/download/resource?kid=TuWJK/hyuhv3aOL1/oJm6bgkER3A7yKaWbpgNQK
//dn-img-page.kakao.com/download/resource?kid=eVqzx/hyuPJ7xQpy/73pBOQuOQyDLeal4jSvCGK
//dn-img-page.kakao.com/download/resource?kid=xBPLA/hyqDpGhCVG/yTYtVuXNWJpGamiuxFZWK1
//dn-img-page.kakao.com/download/resource?kid=YYGm3/hyqFcGX9dz/KzosKWrc22V0ckWD

### 18.4.1 asyncio.as_completed() 사용하기

In [17]:
import asyncio
from collections import namedtuple, Counter
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tqdm


import aiohttp

DEST_DIR = 'async_downloads/'
IMG_PREFIX = 'async'
CONCURRENT_CNT = 1

Result = namedtuple('Result', 'file filename')


def get_categories():
    Category = namedtuple('Category', 'uid subuid')

    cate10 = [1000, 1001, 69, 115, 116, 119, 112]
    cate11 = [1101, 1000, 86, 120, 89, 117, 87]
    cate22 = [2203, 2201, 203, 201, 2209, 2202]
    cate21 = [2103, 201, 2102]
    cate16 = [1601, 84, 51, 113, 40]

    categories = []
    categories += [Category(uid=10, subuid=subuid) for subuid in cate10]
    categories += [Category(uid=11, subuid=subuid) for subuid in cate11]
    categories += [Category(uid=16, subuid=subuid) for subuid in cate16]
    categories += [Category(uid=21, subuid=subuid) for subuid in cate21]
    categories += [Category(uid=22, subuid=subuid) for subuid in cate22]
    return categories

def strtime(timestamp):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(timestamp))


def save_img(img, filename):
    if img and filename:
        path = os.path.join(DEST_DIR, filename)       
        with open(path, 'wb') as f:
            f.write(img)

async def get_top_banner(category, sem):
    async with sem:
        category_url = f'https://page.kakao.com/main?categoryUid={category.uid}&subCategoryUid={category.subuid}'
        content = await aioget(category_url)

        soup = bs(content, 'html.parser')
        divs = soup.findAll('div', {'class': {'topBanner'}})

        if (len(divs)):
            img_src = divs[0].find('img').get('src')
            file = await aiocontent(f'https:{img_src}')
            save_img(file, f'{IMG_PREFIX}-{category.uid}-{category.subuid}-topbanner.png')
            return Result(file, f'{IMG_PREFIX}-{category.uid}-{category.subuid}-topbanner.png')
        else:
            print(f'no top banner on {category.uid} in {category.subuid}')
            return Result(None, None)


async def download_coro(categories):
    sem = asyncio.Semaphore(CONCURRENT_CNT)
    async with aiohttp.ClientSession() as session:
        tasks = [get_top_banner(cate, sem) for cate in categories]
        to_do_iter = asyncio.as_completed(tasks)
        to_do_iter = tqdm(to_do_iter, total=len(tasks))
        for future in to_do_iter:
            try:
                res = await future
            except FetchError as exc:
                print("error!!")
            else:
                status = res
            print(res.filename)

async def main():
    start = time.time()
    print(f'[{strtime(start)}] start!', )
    await download_coro(get_categories())
    end = time.time()
    elapsed = end - start
    print(f'[{strtime(end)}] topbanner img downloaded in {elapsed:.3}s')

asyncio.run(main())

[2019-06-10 03:23:11] start!


async-16-113-topbanner.png
async-11-120-topbanner.png
async-11-117-topbanner.png
async-11-87-topbanner.png
async-22-203-topbanner.png
async-10-1001-topbanner.png
async-10-69-topbanner.png
async-22-201-topbanner.png
async-22-2209-topbanner.png
async-16-51-topbanner.png
async-21-2102-topbanner.png
async-11-86-topbanner.png
async-16-1601-topbanner.png
no top banner on 22 in 2203
None
async-11-89-topbanner.png
async-21-201-topbanner.png
async-16-84-topbanner.png
async-10-1000-topbanner.png
async-10-116-topbanner.png
async-22-2202-topbanner.png
async-10-115-topbanner.png
async-10-119-topbanner.png
async-10-112-topbanner.png
async-16-40-topbanner.png
async-22-2201-topbanner.png
async-11-1101-topbanner.png
async-21-2103-topbanner.png
async-11-1000-topbanner.png
[2019-06-10 03:23:19] topbanner img downloaded in 8.2s


### 18.4.2 Executor 를 이용해서 이벤트 루프 블로킹 피하기 

In [19]:
import time
import asyncio
from collections import namedtuple, Counter
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tqdm


import aiohttp

DEST_DIR = 'async_downloads/'
IMG_PREFIX = 'async'
CONCURRENT_CNT = 1000

Result = namedtuple('Result', 'file filename')


def get_categories():
    Category = namedtuple('Category', 'uid subuid')

    cate10 = [1000, 1001, 69, 115, 116, 119, 112]
    cate11 = [1101, 1000, 86, 120, 89, 117, 87]
    cate22 = [2203, 2201, 203, 201, 2209, 2202]
    cate21 = [2103, 201, 2102]
    cate16 = [1601, 84, 51, 113, 40]

    categories = []
    categories += [Category(uid=10, subuid=subuid) for subuid in cate10]
    categories += [Category(uid=11, subuid=subuid) for subuid in cate11]
    categories += [Category(uid=16, subuid=subuid) for subuid in cate16]
    categories += [Category(uid=21, subuid=subuid) for subuid in cate21]
    categories += [Category(uid=22, subuid=subuid) for subuid in cate22]
    return categories

def strtime(timestamp):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(timestamp))


def save_img(img, filename):
    if img and filename:
        path = os.path.join(DEST_DIR, filename)       
        with open(path, 'wb') as f:
            f.write(img)

async def get_top_banner(category, sem):
    async with sem:
        category_url = f'https://page.kakao.com/main?categoryUid={category.uid}&subCategoryUid={category.subuid}'
        content = await aioget(category_url)

        soup = bs(content, 'html.parser')
        divs = soup.findAll('div', {'class': {'topBanner'}})

        if (len(divs)):
            img_src = divs[0].find('img').get('src')
            file = await aiocontent(f'https:{img_src}')
            ### 이부분이 변경됨 
            loop = asyncio.get_event_loop()
            loop.run_in_executor(None, save_img, file, f'{IMG_PREFIX}-{category.uid}-{category.subuid}-topbanner.png')
            return Result(file, f'{IMG_PREFIX}-{category.uid}-{category.subuid}-topbanner.png')
        else:
            print(f'no top banner on {category.uid} in {category.subuid}')
            return Result(None, None)


async def download_coro(categories):
    sem = asyncio.Semaphore(CONCURRENT_CNT)
    async with aiohttp.ClientSession() as session:
        tasks = [get_top_banner(cate, sem) for cate in categories]
        to_do_iter = asyncio.as_completed(tasks)
        to_do_iter = tqdm(to_do_iter, total=len(tasks))
        for future in to_do_iter:
            try:
                res = await future
            except FetchError as exc:
                print("error!!")
            else:
                status = res
            print(res.filename)

async def main():
    start = time.time()
    print(f'[{strtime(start)}] start!', )
    await download_coro(get_categories())
    end = time.time()
    elapsed = end - start
    print(f'[{strtime(end)}] topbanner img downloaded in {elapsed:.3}s')

asyncio.run(main())

[2019-06-10 03:24:39] start!


no top banner on 22 in 2203
None
async-16-1601-topbanner.png
async-11-1000-topbanner.png
async-16-51-topbanner.png
async-10-1000-topbanner.png
async-16-84-topbanner.png
async-10-115-topbanner.png
async-11-89-topbanner.png
async-22-201-topbanner.png
async-11-86-topbanner.png
async-10-116-topbanner.png
async-21-201-topbanner.png
async-11-87-topbanner.png
async-22-2202-topbanner.png
async-22-203-topbanner.png
async-11-117-topbanner.png
async-11-120-topbanner.png
async-10-112-topbanner.png
async-11-1101-topbanner.png
async-16-40-topbanner.png
async-16-113-topbanner.png
async-10-119-topbanner.png
async-22-2209-topbanner.png
async-10-1001-topbanner.png
async-21-2103-topbanner.png
async-21-2102-topbanner.png
async-10-69-topbanner.png
async-22-2201-topbanner.png
[2019-06-10 03:24:41] topbanner img downloaded in 2.25s


## 18.5 콜백에서 Future 와 코루틴으로 

In [21]:
def stage1(res1):
    print('stage1')
    res2 = step1(res1)
    api_call2(res2, stage2)

def stage2(res2):
    print('stage2')
    req3 = step2(res2)
    api_call3(req3, stage3)

def stage3(res3):
    print('stage3')
    step3(res3)


def step1(res):
    print('step1')
    print(res)

def step2(res):
    print('step2')
    print(res)

def step3(res):
    print('step3')
    print(res)
    
def api_call1(req, stage):
    print('api_call_1')
    stage(req)

def api_call2(req, stage):
    print('api_call_2')
    stage(req)

def api_call3(req, stage):
    print('api_call_3')
    stage(req)

    
api_call1('req1', stage1)


api_call_1
stage1
step1
req1
api_call_2
stage2
step2
None
api_call_3
stage3
step3
None


In [22]:
import asyncio 

def step1(res):
    print('step1', res)
    return res

def step2(res):
    print('step2' ,res)
    return res

def step3(res):
    print('step3', res)
    return res
    
async def api_call1(req):
    print('api_call_1', req)
    return req

async def api_call2(req):
    print('api_call_2', req)
    return req


async def api_call3(req):
    print('api_call_3', req)
    return req


async def async_call(req1):
    res1 = await api_call1(req1)
    req2 = step1(res1)
    res2 = await api_call2(req2)
    req3 = step2(res2)
    res3 = await api_call3(req3)
    step3(res3)

asyncio.run(async_call('reqeust'))

api_call_1 reqeust
step1 reqeust
api_call_2 reqeust
step2 reqeust
api_call_3 reqeust
step3 reqeust


### 예제 18-14 tcp_charfinder.py : asyncio_start_server() 를 사용한 간단한 TCP 서버

아래 소스는 실제로 서버를 구동시켜야 하는 작업이라 jupyter에서는 실행되지 않으므로, 
콘솔에서 테스트 해보자

In [23]:
import sys
import asyncio

from charfinder import UnicodeNameIndex

CRLF = b'\r\n'
PROMPT = b'?> '

index = UnicodeNameIndex()

async def handle_queries(reader, writer): 
    """
    asyncio.StreamReader 와 asyncio.StreamWriter 객체를 파라메터로 받는다.
    StreamWriter.write() 는 일반함수라서 블로킹된다. 
    
    StreamWriter.write(), readline() 은 코루틴으로 구현되어서 비동기로 동작한다. 
    StreamWriter.drain() 메서드는 출력 버퍼를 플러시한다. 
    클라이언트에서 오는 여러쿼리를 처리하므로 함수명이 복수이다. 
    
    아래 문서 참고 
    https://docs.python.org/ko/3/library/asyncio-stream.html
    """
    while True:
        writer.write(PROMPT)
        await writer.drain()
        data = await reader.readline()
        try:
            query = data.decode().strip()
        except UnicodeDecodeError:
            query = '\x00'
        client = writer.get_extra_info('peername')
        print('Received from {}: {!r}'.format(client, query))
        if query:
            if ord(query[:1]) < 32:
                break
            lines = list(index.find_description_strs(query))
            if lines:
                writer.writelines(line.encode() + CRLF for line in lines)
            writer.write(index.status(query, len(lines)).encode() + CRLF)
            
            await writer.drain()
            print("Sent {} results.".format(len(lines)))
            print('Close the clien socket')
            writer.close()

def main(address='127.0.0.1', port=2323):
    port=int(port)
    loop = asyncio.get_event_loop()
    server_coro = asyncio.start_server(handle_queries, address, port, loop=loop)
    server = loop.run_until_complete(server_coro)
    host = server.sockets[0].getsocketname()
    print(f'서빙중 {host}. CTRL-C 를 누르면 중단합니다. ')
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        pass
    
    print('서버 셧다운')
    server.close()
    loop.run_until_complte(server.wait_closed())
    loop.close()

asyncio.run(main())

RuntimeError: There is no current event loop in thread 'MainThread'.

## char 예제

In [ ]:
import pytest

from charfinder import UnicodeNameIndex, tokenize, sample_chars, query_type
from unicodedata import name


@pytest.fixture
def sample_index():
    return UnicodeNameIndex(sample_chars)


@pytest.fixture(scope="module")
def full_index():
    return UnicodeNameIndex()


def test_query_type():
    assert query_type('blue') == 'NAME'


def test_tokenize():
    assert list(tokenize('')) == []
    assert list(tokenize('a b')) == ['A', 'B']
    assert list(tokenize('a-b')) == ['A', 'B']
    assert list(tokenize('abc')) == ['ABC']
    assert list(tokenize('café')) == ['CAFÉ']


def test_index():
    sample_index = UnicodeNameIndex(sample_chars)
    assert len(sample_index.index) == 9


def test_find_word_no_match(sample_index):
    res = sample_index.find_chars('qwertyuiop')
    assert len(res.items) == 0


def test_find_word_1_match(sample_index):
    res = [(ord(char), name(char))
           for char in sample_index.find_chars('currency').items]
    assert res == [(8352, 'EURO-CURRENCY SIGN')]


def test_find_word_1_match_character_result(sample_index):
    res = [name(char) for char in
           sample_index.find_chars('currency').items]
    assert res == ['EURO-CURRENCY SIGN']


def test_find_word_2_matches(sample_index):
    res = [(ord(char), name(char))
           for char in sample_index.find_chars('Euro').items]
    assert res == [(8352, 'EURO-CURRENCY SIGN'),
                   (8364, 'EURO SIGN')]


def test_find_2_words_no_matches(sample_index):
    res = sample_index.find_chars('Euro letter')
    assert res.count == 0


def test_find_2_words_no_matches_because_one_not_found(sample_index):
    res = sample_index.find_chars('letter qwertyuiop')
    assert res.count == 0


def test_find_2_words_1_match(sample_index):
    res = sample_index.find_chars('sign dollar')
    assert res.count == 1


def test_find_2_words_2_matches(sample_index):
    res = sample_index.find_chars('latin letter')
    assert res.count == 2


def test_find_chars_many_matches_full(full_index):
    res = full_index.find_chars('letter')
    assert res.count > 7000


def test_find_1_word_1_match_full(full_index):
    res = [(ord(char), name(char))
           for char in full_index.find_chars('registered').items]
    assert res == [(174, 'REGISTERED SIGN')]


def test_find_1_word_2_matches_full(full_index):
    res = full_index.find_chars('rook')
    assert res.count == 2


def test_find_3_words_no_matches_full(full_index):
    res = full_index.find_chars('no such character')
    assert res.count == 0


def test_find_with_start(sample_index):
    res = [(ord(char), name(char))
           for char in sample_index.find_chars('sign', 1).items]
    assert res == [(8352, 'EURO-CURRENCY SIGN'), (8364, 'EURO SIGN')]


def test_find_with_stop(sample_index):
    res = [(ord(char), name(char))
           for char in sample_index.find_chars('sign', 0, 2).items]
    assert res == [(36, 'DOLLAR SIGN'), (8352, 'EURO-CURRENCY SIGN')]


def test_find_with_start_stop(sample_index):
    res = [(ord(char), name(char))
           for char in sample_index.find_chars('sign', 1, 2).items]
assert res == [(8352, 'EURO-CURRENCY SIGN')]